# 02 - Strategic Analysis

This notebook analyzes feature-engineered data to identify patterns and insights for model selection.

**Course:** DATA 230 (Data Visualization) at SJSU


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Load feature-engineered data
df = pd.read_csv('../data/ml/feature_engineered_data.csv')
print(f"Loaded {len(df)} records with {len(df.columns)} features")
